<a href="https://colab.research.google.com/github/thatbeer/AI4ENG/blob/main/ECEF2XYZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##installing extra module

In [2]:
pip install pyproj

     |████████████████████████████████| 6.3 MB 4.2 MB/s 


In [3]:
pip install pygeodesy

     |████████████████████████████████| 737 kB 4.3 MB/s 


In [4]:
pip install geodepy

     |████████████████████████████████| 49 kB 2.3 MB/s 


In [5]:
"""
%   Created on Thu Sep 19 13:21:19 2021
%   @author: Chaiyut Charoenphon
%   Course : Geodetic Survey 2108415
  Ref:
        Geoscience Australia - Python Geodesy Package
        Geodesy Module
        https://github.com/GeoscienceAustralia/GeodePy/blob/master/geodepy/geodesy.py
"""
from pyproj import Geod
import pyproj
import pygeodesy
import geodepy
import pygeodesy.dms as dms
from math import degrees, radians, sqrt, sin, cos, tan, asin, acos, atan, atan2
from geodepy.constants import grs80, utm, wgs84
from geodepy.convert import geo2grid, grid2geo, angular_typecheck

In [6]:
from pygeodesy.ellipsoidalKarney import LatLon

## Function

In [7]:

############################################################
def dd2dms(dd, PREC=5):
    #arcsecond is 5-digit decimal fraction
    return dms.toDMS(dd,prec=5)
def dms2dd(dms_str,SEP=' '):
    return dms.parseDMS(dms_str, sep=SEP)
    
#คำนวณรัศมีโลก ณ Latitude โดยประมาณ
def radius (B):
    # input:  B is Latitude in Degree unit
    # output: (Approximate) radius in m. unit (Approximate)
    B = radians(B) #converting into radians
    # WGS84
    a = 6378.137  #Radius at sea level at equator in m. unit
    b = 6356.752314245  #Radius at poles in m. unit
    c = (a**2*cos(B))**2
    d = (b**2*sin(B))**2
    e = (a*cos(B))**2
    f = (b*sin(B))**2
    R = sqrt((c+d)/(e+f))*1000.0
    return R

#แปลงค่าพิกัด xyz เป็น ละติจูด ลองจิจูด และ ความสูงเหนือทรงรี  หน่วยดีกรี
def xyz2lla(x,y,z):
    ecef = '+proj=geocent +ellps=WGS84' # Cartisian   
    lla = "epsg:4326" # WGS84 Geodetic    
    transproj = pyproj.Transformer.from_crs(ecef,lla)
    lat,lon,ell_h = transproj.transform(x,y,z,radians=False)
    return lat,lon,ell_h

#แปลงค่าพิกัด ละติจูด ลองจิจูด และ ความสูงเหนือทรงรี   เป็นพิกัด UTM Zone 47N
def lla2utm(lat,lon,ell_h,zone):
    '''
        I = lat ,long unit Degree
        I = ell_h unit metre
        O = E,N,h unit metre
    '''
    lla = "epsg:4326" # WGS84 Geodetic
    utm = "epsg:326" + zone # WGS84 UTM Zone 47N  
    transproj = pyproj.Transformer.from_crs(lla,utm,always_xy=True) # always_xy หมายถึงให้เรียง lon,lat
    E,N,h = transproj.transform(lon,lat,ell_h,radians=False)
    return E,N,h

#คำนวนสเกลแฟกเตอร์ในระบบพิกัด UTM โดยระบบพิกัดละติจูด ลองจิจูด และ โซน
def utm_scale_factor(lat,lon,zone):
    '''
        I = lat ,long unit Degree
        I = zone (UTM)
        O = scaleX, scaleY, scaleArea, mer_conv
    '''
    prj = pyproj.Proj('epsg:326' + zone)
    fc = prj.get_factors(lon,lat,radians=False)
    scaleX = fc.parallel_scale
    scaleY = fc.meridional_scale
    scaleArea = fc.areal_scale 
    mer_conv = fc.meridian_convergence
    return scaleX, scaleY, scaleArea, mer_conv

#คำนวนความสูงเหนือระดับน้ำทะเลปานกลาง อ้างอิงแบบจำลอง TGM2017 
def MSL(lat,lon,h):
    #  Download: tgm2017-1_2.pgm size 469 MB
    #  https://www.priabroy.name/archives/sdm_downloads/tgm2017-1
    geoid_model = r'C:\Users\Chaiyut\OneDrive - Faculty of Engineering, Chulalongkorn University\MyJob\GISDATA\TGM2017\tgm2017-1_2.pgm'
    geoid_interpolator = pygeodesy.GeoidKarney(geoid_model)
    # Get the geoid height
    pos = LatLon(lat, lon)
    N = geoid_interpolator(pos)
    return h-N
def h(lat,lon,H):
    #  Download: tgm2017-1_2.pgm size 469 MB
    #  https://www.priabroy.name/archives/sdm_downloads/tgm2017-1
    geoid_model = r'/content/tgm2017-1.pgm'
    geoid_interpolator = pygeodesy.GeoidKarney(geoid_model)
    # Get the geoid height
    pos = LatLon(lat, lon)
    N = geoid_interpolator(pos)
    return H+N 

#แปลงค่าพิกัด xyz เป็นพิกัด Lambert Conformal Conic (LCC)
def xyz2LCC(x,y,z):   
    #Lambert Conformal Conic 1 standard parallel Conversion
    llc_1sp   = '+proj=lcc +lon_0=99 +lat_0=0 +x_0=500000 +y_0=0 +ellps=WGS84 +lat_1=33'
    #Lambert Conformal Conic conversion with scale factor
    llc_2sp_1 = '+proj=lcc +lon_0=99 +lat_0=0 +x_0=500000 +y_0=0 +ellps=WGS84 +k=0.9996'
    #Lambert Conformal Conic 2 standard parallel Conversion
    llc_2sp_2 = '+proj=lcc +lon_0=99 +lat_0=0 +x_0=500000 +y_0=0 +ellps=WGS84 +lat_1=9 +lat_2=12'
    
    ecef = {"proj":'geocent', "ellps":'WGS84'}
    transproj = pyproj.Transformer.from_crs(ecef,llc_1sp)
    E,N,h = transproj.transform(x,y,z)
    return (E,N,h)

def utm2lla(E,N,h,zone):   
    utm = "epsg:326" + zone # WGS84 UTM Zone 47N  
    lla = "epsg:4326" # WGS84 Geodetic 
    transproj = pyproj.Transformer.from_crs(utm,lla) # always_xy หมายถึงให้เรียง lon,lat
    lat,lon,h = transproj.transform(E,N,h,radians=False)
    return lat,lon,h
def utm2xyz(E,N,h,zone):
    utm = "epsg:326" + zone # WGS84 UTM Zone 47N  
    ecef = {"proj":'geocent', "ellps":'WGS84'}   
    transproj = pyproj.Transformer.from_crs(utm,ecef,always_xy=True) # always_xy หมายถึงให้เรียง lon,lat
    x,y,z = transproj.transform(E,N,h)
    return x,y,z
def lla2xyz(lat,lon,h):
    lla = "epsg:4326" # WGS84 Geodetic
    ecef = {"proj":'geocent', "ellps":'WGS84'}
    transproj = pyproj.Transformer.from_crs(lla,ecef,always_xy=True) # always_xy หมายถึงให้เรียง lon,lat
    x,y,z = transproj.transform(lon,lat,h,radians=False)
    return x,y,z    
def xyz2utm(x,y,z,zone):
    utm = "epsg:326" + zone # WGS84 UTM Zone 47N  
    ecef = {"proj":'geocent', "ellps":'WGS84'}
    transproj = pyproj.Transformer.from_crs(ecef,utm) # always_xy หมายถึงให้เรียง lon,lat
    E,N,h = transproj.transform(x,y,z)
    return E,N,h
def utm2tm(E,N,h,from_crs):
    utm = f"epsg:{from_crs}"  # UTM EPSG:___
    tm = '+proj=tmerc +lat_0=0 +lon_0=99.417 +k=1.00 +x_0=500000 +y_0=0.0 +ellis = WGS84 +units=m'
    transproj = pyproj.Transformer.from_crs(utm,tm,always_xy=True) 
    En,Nn,h = transproj.transform(E,N,h)
    return En,Nn,h 

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def line_sf(zone1, east1, north1, zone2, east2, north2,
            hemisphere='south', ellipsoid=grs80, projection=utm):
    """
    Computes Line Scale Factor for a pair of Transverse Mercator Coordinates
    Ref: Deakin 2010, Traverse Computations on the Ellipsoid and on the
    Universal Transverse Mercator Projection, pp 35
    http://www.mygeodesy.id.au/documents/Trav_Comp_V2.1.pdf
    :param zone1: Station 1 Zone Number - 1 to 60
    :param east1: Station 1 Easting (m, within 3330km of Central Meridian)
    :param north1: Station 1 Northing (m, 0 to 10,000,000m)
    :param zone2: Station 2 Zone Number - 1 to 60
    :param east2: Station 2 Easting (m, within 3330km of Central Meridian)
    :param north2: Station 2 Northing (m, 0 to 10,000,000m)
    :param hemisphere: String - 'North' or 'South'(default)
    :param ellipsoid: Ellipsoid Object (default GRS80)
    :param projection: Projection Object (default Universal Transverse Mercator)
    :return: Line Scale Factor (relative to Zone 1 if different zones
    are entered)
    """
    # Re-project cross-zone coordinate to same UTM zone
    if zone1 != zone2:
        # Re-project Station 2 Coordinates to same zone as Station 1
        stn2_geo = grid2geo(zone2, east2, north2, hemisphere, ellipsoid)
        stn2_zone1 = geo2grid(stn2_geo[0], stn2_geo[1], zone1, ellipsoid)
        zone2 = stn2_zone1[1]
        east2 = stn2_zone1[2]
        north2 = stn2_zone1[3]

    # Comute easting distances from Central Meridian
    eastofcm1 = east1 - projection.falseeast
    eastofcm2 = east2 - projection.falseeast

    # Compute Mean Latitude
    lat1 = grid2geo(zone1, east1, north1, hemisphere, ellipsoid)[0]
    lat2 = grid2geo(zone2, east2, north2, hemisphere, ellipsoid)[0]
    lat_mean = (lat1 + lat2) / 2

    # Compute Line Scale Factor (Deakin 2010 Eq. 13)
    r_sq_m = (rho(lat_mean, ellipsoid) *
              nu(lat_mean, ellipsoid) *
              projection.cmscale ** 2)
    k1 = ((eastofcm1 ** 2 + eastofcm1 * eastofcm2 + eastofcm2 ** 2) /
          (6 * r_sq_m))
    k2 = ((eastofcm1 ** 2 + eastofcm1 * eastofcm2 + eastofcm2 ** 2) /
          (36 * r_sq_m))
    return projection.cmscale * (1 + k1 * (1 + k2))
    
def rho(lat, ellipsoid=grs80):
    """
    symbol = 
    Return the radius of curvature of the ellipsoid in the meridian plane
    (rho) at a given latitude
    :param lat: latitude in decimal degrees
    :param ellipsoid: Ellipsoid Object
    :return: rho at specified latitude
    """
    return ((ellipsoid.semimaj * (1 - ellipsoid.ecc1sq)) /
            (1 - ellipsoid.ecc1sq * (sin(radians(lat)) ** 2)) ** 1.5)

def nu(lat, ellipsoid=grs80):
    """
    symbol = v
    Return the radius of curvature of the ellipsoid in the prime vertical plane
    (nu) at a given latitude
    :param lat: latitude in decimal degrees
    :param ellipsoid: Ellipsoid Object
    :return: nu at specified latitude
    """
    return (ellipsoid.semimaj / sqrt(1 - ellipsoid.ecc1sq * (sin(radians(lat)) ** 2)))   

def Rns(lat, az12, ellipsoid=grs80):
    """
    Return the radius of  point 1 of the normal section in the direction Az12
    :param lat: latitude in decimal degrees
    :param az12: Azimuth Point 1 to Point 2 in decimal degrees
    :param ellipsoid: Ellipsoid Object default grs80
    :return: radius at specified latitude in m unit
    nu = v
    """  
    _rho = rho(lat,ellipsoid)
    _nu  = nu(lat,ellipsoid)
    return (_rho*_nu) / (_rho*(sin(radians(az12))**2) + _nu*(cos(radians(az12)))**2)   
   
def GrdD_to_ELL(R,D,h1,h2):
    '''
        Convert Ground distance to ellipsoidal distance in m unit
        input:
            D is the ground distance in m. unit
            h1 and h2 are the ellipsoidal height (m. unit) at P1 and P2, respectively.
        output:
            ellipsoidal distance m. unit
    '''
    dh = h2-h1
    c = R*sqrt((D**2-dh**2)/((R+h1)*(R+h2)))
    #c = R*D/(R+(h1+h2)*0.5)
    print(c)
    s = 2.0*R*asin(c/(2.0*R))
    return s

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
'''    
geom = Geod(ellps="WGS84")
lat = dms2dd('-38 09 05.2006')
lon = dms2dd('144 36 43.6942')
az = dms2dd('127 14 29.633')
gs = 37176.169
endlon, endlat, backaz = geom.fwd(lon, lat, az, gs)
print(f"{dd2dms(endlon)} {dd2dms(endlat)} {dd2dms(backaz+360)}")
'''

geom = Geod(ellps="GRS80")
lat = dms2dd('-37 17 49.7306')
lon = dms2dd('143 59 03.1691')
endlat = dms2dd('-37 39 10.1563')
endlon = dms2dd('143 55 35.3839')

az12,az21,dist = geom.inv(lon,lat,endlon,endlat)
#print(f"{dd2dms(az12+360)} {dd2dms(az21)} {dist:.3f}")


lat = dms2dd('-37 57 03.7047')
lon = dms2dd('144 25 29.5333')
az = dms2dd('143 35 52.834')
gs = 27659.183
endlon, endlat, backaz = geom.fwd(lon, lat, az, gs)
print(f"{dd2dms(endlon)} {dd2dms(endlat)}")

'''

GPS01= 543302.071,1561292.574,5.276
GPS02= 543187.329,1560825.733,4.288
g01 = utm2lla(GPS01[0],GPS01[1],GPS01[2],'47')
g02 = utm2lla(GPS02[0],GPS02[1],GPS02[2],'47')
geom = Geod(ellps="WGS84")
az12,az21,dist = geom.inv(g01[1],g01[0],g02[1],g02[0])
print(dd2dms(az21))
'''

144°36′43.69427″ -38°09′05.20064″


'\n\nGPS01= 543302.071,1561292.574,5.276\nGPS02= 543187.329,1560825.733,4.288\ng01 = utm2lla(GPS01[0],GPS01[1],GPS01[2],\'47\')\ng02 = utm2lla(GPS02[0],GPS02[1],GPS02[2],\'47\')\ngeom = Geod(ellps="WGS84")\naz12,az21,dist = geom.inv(g01[1],g01[0],g02[1],g02[0])\nprint(dd2dms(az21))\n'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def MSL(lat,lon,h):
    #  Download: tgm2017-1_2.pgm size 469 MB
    #  https://www.priabroy.name/archives/sdm_downloads/tgm2017-1
    geoid_model = '/content/tgm2017-1.pgm'
    geoid_interpolator = pygeodesy.GeoidKarney(geoid_model)
    # Get the geoid height
    pos = LatLon(lat, lon)
    N = geoid_interpolator(pos)
    return h-N
def ellip_h(lat,lon,H):
    #  Download: tgm2017-1_2.pgm size 469 MB
    #  https://www.priabroy.name/archives/sdm_downloads/tgm2017-1
    geoid_model = '/content/drive/MyDrive/tgm2017/tgm2017-1_2.pgm'
    geoid_interpolator = pygeodesy.GeoidKarney(geoid_model)
    # Get the geoid height
    pos = LatLon(lat, lon)
    N = geoid_interpolator(pos)
    return H+N 


In [10]:
E,N,H = (669868.188	,1522532.81,	0.47)
lat , lon ,H = utm2lla(E,N,H,'47')
h = ellip_h(lat,lon,H)
h

-29.907592287777423

### TEst

In [15]:
dd10 = [[667522.37	,1522086.727	,0.109],
[667522.372	,1522086.735	,0.133],
[667522.372	,1522086.734,	0.119],
[667522.373,	1522086.735	,0.101],
[667522.372	,1522086.734	,0.097],
[670190.083	,1525123.837,	0.943],
[670190.08,	1525123.834	,0.944],
[670190.083,	1525123.835,	0.933],
[670190.08	,1525123.834,	0.944],
[670190.089	,1525123.847,	0.93],
[669868.196	,1522532.821,	0.453],
[669868.196,	1522532.821	,0.453],
[669868.196,	1522532.821,	0.453],
[669868.192	,1522532.814	,0.461],
[669868.188	,1522532.81,	0.47]]
Xs = []
Ys = []
Zs = []
for i in range(15):
  #Xs = []
  #Ys = []
  #Zs = []
  E = dd10[i][0]
  N = dd10[i][1]
  H = dd10[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)


[-1134423.3902086022, -1134423.3961454234, -1134423.3936930604, -1134423.3914383247, -1134423.389780836, -1136933.6847599417, -1136933.682100108, -1136933.6830523403, -1136933.682100108, -1136933.6879699405, -1136713.6596726622, -1136713.6596726622, -1136713.6596726622, -1136713.657425538, -1136713.6552448666]


## DICT

In [18]:
import pandas as pd
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
e = df.to_csv('dd10.csv',index=False)

##DD01


In [19]:
dd10 = [[669754.355	,1525372.418,	0.967],
[669754.354	,1525372.418,	0.969],
[669754.353	,1525372.417,	0.97],
[669754.352	,1525372.417,	0.969],
[669754.352	,1525372.417,	0.966],
[669506.646	,1525015.437,	0.824],
[669506.645	,1525015.437,	0.818],
[669506.643	,1525015.438,	0.817],
[669506.644	,1525015.44,	0.827],
[669506.643	,1525015.44	,0.834],
[669316.234	,1525064.915,	0.995],
[669316.236	,1525064.915,	0.995],
[669316.237	,1525064.915,	1.002],
[669316.236	,1525064.916,	0.999],
[669316.237	,1525064.916,	1.005],
[669117.939	,1525069.49,	0.924],
[669117.943	,1525069.487,	0.924],
[669117.941	,1525069.487,	0.934],
[669117.941,	1525069.487,	0.932],
[669117.939,	1525069.486,	0.928],
[668847.048,	1525088.555,	0.418],
[668847.054,	1525088.55,	0.412],
[668847.057,	1525088.548,	0.402],
[668847.052,	1525088.549	,0.387],
[668847.053,	1525088.55	,0.39]]


Xs = []
Ys = []
Zs = []
for i in range(25):
  #Xs = []
  #Ys = []
  #Zs = []
  E = dd10[i][0]
  N = dd10[i][1]
  H = dd10[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df

[-1136495.948993733, -1136495.9483666979, -1136495.9475988254, -1136495.9464373344, -1136495.945902875, -1136265.659100118, -1136265.657048067, -1136265.6548659515, -1136265.6575558607, -1136265.6578193314, -1136076.602084649, -1136076.604051357, -1136076.6062813224, -1136076.6047263998, -1136076.6067782773, -1135881.4240972034, -1135881.4281425597, -1135881.4279564074, -1135881.4276002937, -1135881.424958655, -1135614.2383394637, -1135614.2433581147, -1135614.2446026837, -1135614.2369783118, -1135614.2384584183]


,X,Y,Z
0,-1.136496e+06,6.090234e+06,1.510728e+06
1,-1.136496e+06,6.090234e+06,1.510728e+06
2,-1.136496e+06,6.090234e+06,1.510728e+06
3,-1.136496e+06,6.090234e+06,1.510728e+06
4,-1.136496e+06,6.090234e+06,1.510728e+06
5,-1.136266e+06,6.090363e+06,1.510383e+06
6,-1.136266e+06,6.090363e+06,1.510383e+06
7,-1.136266e+06,6.090363e+06,1.510383e+06
8,-1.136266e+06,6.090363e+06,1.510383e+06
9,-1.136266e+06,6.090363e+06,1.510383e+06


In [20]:
e = df.to_csv('dd01.csv',index=False)

dd07

In [21]:
dd10 = [[668167.527,	1521954.849,	0.397],
[668167.522,	1521954.853,	0.392],
[668167.519,	1521954.858,	0.408],
[668167.518,	1521954.854,	0.416],
[668167.531,	1521954.849,	0.429],
[668118.906,	1522471.212,	0.759],
[668118.907,	1522471.217,	0.775],
[668118.905,	1522471.217,	0.774],
[668118.901,	1522471.223,	0.786],
[668118.905,	1522471.228,	0.799],
[667522.372,	1522086.734,	0.097],
[667522.373,	1522086.735,	0.101],
[667522.372,	1522086.734,	0.119],
[667522.372,	1522086.735,	0.133],
[667522.37,	1522086.727,	0.109],
[667877.536,	1523058.528,	0.155],
[667877.535,	1523058.526,	0.155],
[667877.538,	1523058.528,	0.15],
[667877.54,	1523058.525,	0.146],
[667877.544,	1523058.524,	0.14]]


Xs = []
Ys = []
Zs = []
for i in range(20):
  #Xs = []
  #Ys = []
  #Zs = []
  E = dd10[i][0]
  N = dd10[i][1]
  H = dd10[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df

[-1135062.7987613303, -1135062.792805814, -1135062.7925163463, -1135062.7931053124, -1135062.8083886034, -1134995.8214976871, -1134995.8251415472, -1134995.8229968487, -1134995.8209748345, -1134995.827035114, -1134423.389780836, -1134423.3914383247, -1134423.3936930604, -1134423.3961454234, -1134423.3902086022, -1134736.473813297, -1134736.4729044146, -1134736.4748907078, -1134736.47625777, -1134736.479161357]


,X,Y,Z
0,-1.135063e+06,6.091326e+06,1.507418e+06
1,-1.135063e+06,6.091326e+06,1.507418e+06
2,-1.135063e+06,6.091326e+06,1.507418e+06
3,-1.135063e+06,6.091326e+06,1.507418e+06
4,-1.135063e+06,6.091326e+06,1.507418e+06
5,-1.134996e+06,6.091214e+06,1.507920e+06
6,-1.134996e+06,6.091214e+06,1.507920e+06
7,-1.134996e+06,6.091214e+06,1.507920e+06
8,-1.134996e+06,6.091214e+06,1.507920e+06
9,-1.134996e+06,6.091214e+06,1.507920e+06


In [22]:
e = df.to_csv('dd07.csv',index=False)

dd09

In [23]:
dd10 = [[669182.556,	1521763.182,	0.583],
[669182.546,	1521763.179,	0.606],
[669182.546,	1521763.179,	0.604],
[669182.548,	1521763.182,	0.598],
[669182.548,	1521763.181,	0.604],
[668997.117,	1521838.425,	0.405],
[66897.118,	1521838.426,	0.401],
[668997.119,	1521838.426,	0.399],
[668997.119,	1521838.426,	0.398],
[668997.114,	1521838.427,	0.397],
[669330.087,	1521851.819,	0.553],
[669330.85,	1521851.82,	0.551],
[669330.086,	1521851.822,	0.555],
[669330.083,	1521851.823,	0.55],
[669330.083,	1521851.823,	0.552],
[669186.749,	1521328.925,	0.621],
[669186.754,	1521328.929,	0.619],
[669186.758,	1521328.93,	0.626],
[669186.76,	1521328.929,	0.635],
[669186.755,	1521328.928,	0.641]]



Xs = []
Ys = []
Zs = []
for i in range(20):
  #Xs = []
  #Ys = []
  #Zs = []
  E = dd10[i][0]
  N = dd10[i][1]
  H = dd10[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df

[-1136068.1197707027, -1136068.114144796, -1136068.1137886236, -1136068.1145751402, -1136068.1156808846, -1135882.9336551987, -539766.0423882832, -1135882.9345163517, -1135882.9343382944, -1135882.9292061944, -1136209.8899446495, -1136210.639848999, -1136209.8892058278, -1136209.8853279997, -1136209.8856842169, -1136088.413551338, -1136088.4179630785, -1136088.4231059137, -1136088.426712648, -1136088.422901617]


,X,Y,Z
0,-1.136068e+06,6.091187e+06,1.507226e+06
1,-1.136068e+06,6.091187e+06,1.507226e+06
2,-1.136068e+06,6.091187e+06,1.507226e+06
3,-1.136068e+06,6.091187e+06,1.507226e+06
4,-1.136068e+06,6.091187e+06,1.507226e+06
5,-1.135883e+06,6.091203e+06,1.507300e+06
6,-5.397660e+05,6.173361e+06,1.504334e+06
7,-1.135883e+06,6.091203e+06,1.507300e+06
8,-1.135883e+06,6.091203e+06,1.507300e+06
9,-1.135883e+06,6.091203e+06,1.507300e+06


In [24]:
e = df.to_csv('dd09.csv',index=False)

GCP control point


In [144]:
gcp = 
Xs = []
Ys = []
Zs = []
for i in range(3):
  #Xs = []
  #Ys = []
  #Zs = []
  E = gcp[i][0]
  N = gcp[i][1]
  H = gcp[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df

SyntaxError: ignored

In [26]:
gcp = [[667522.426,	1522086.7,	0.13],
[670190.11,	1525123.734,	1.132],
[669868.18,	1522532.79,	0.609]]
Xs = []
Ys = []
Zs = []
for i in range(3):
  #Xs = []
  #Ys = []
  #Zs = []
  E = gcp[i][0]
  N = gcp[i][1]
  H = gcp[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df
##dd10

[-1134423.4500192902, -1136933.7488362715, -1136713.6728911358]


,X,Y,Z
0,-1.134423e+06,6.091412e+06,1.507550e+06
1,-1.136934e+06,6.090214e+06,1.510484e+06
2,-1.136714e+06,6.090882e+06,1.507969e+06


In [27]:
e = df.to_csv('gcp10.csv',index=False)

In [28]:
gcp = [[669754.295,	1525372.4,	1.035],
[669506.646,	1525015.446,	0.802],
[669316.163,	1525064.872,	1.059],
[669117.856,	1525069.607,	1.016],
[668846.974,	1525088.51,	0.272]]
Xs = []
Ys = []
Zs = []
for i in range(5):
  #Xs = []
  #Ys = []
  #Zs = []
  E = gcp[i][0]
  N = gcp[i][1]
  H = gcp[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df
#dd01

[-1136495.902779416, -1136265.6548458005, -1136076.5452683093, -1135881.3544947756, -1135614.1412588344]


,X,Y,Z
0,-1.136496e+06,6.090234e+06,1.510728e+06
1,-1.136266e+06,6.090363e+06,1.510383e+06
2,-1.136077e+06,6.090386e+06,1.510432e+06
3,-1.135881e+06,6.090421e+06,1.510438e+06
4,-1.135614e+06,6.090465e+06,1.510458e+06


In [29]:
e = df.to_csv('gcp01.csv',index=False)

In [30]:
gcp = [[668167.516,	1521954.873,	0.498],
[668118.791,	1522471.219,	0.765],
[667522.315,	1522086.742,	0.206],
[667877.431,	1523058.472,	0.254]]

Xs = []
Ys = []
Zs = []
for i in range(4):
  #Xs = []
  #Ys = []
  #Zs = []
  E = gcp[i][0]
  N = gcp[i][1]
  H = gcp[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df
##d07

[-1135062.805021278, -1134995.7092145258, -1134423.3528118918, -1134736.390252765]


,X,Y,Z
0,-1.135063e+06,6.091326e+06,1.507418e+06
1,-1.134996e+06,6.091214e+06,1.507920e+06
2,-1.134423e+06,6.091412e+06,1.507550e+06
3,-1.134736e+06,6.091119e+06,1.508492e+06


In [31]:
e = df.to_csv('gcp07.csv',index=False)

In [32]:
gcp = [[669182.563,	1521763.106,	0.884],
[668997.133,	1521838.43,	0.395],
[669330.093,	1521851.828,	0.522],
[669186.764,	1521328.937,	0.744]]

Xs = []
Ys = []
Zs = []
for i in range(4):
  #Xs = []
  #Ys = []
  #Zs = []
  E = gcp[i][0]
  N = gcp[i][1]
  H = gcp[i][2]
  lat , lon ,H = utm2lla(E,N,H,'47')
  h = ellip_h(lat,lon,H)
  X,Y,Z = lla2xyz(lat,lon,h)
  Xs.append(X)
  Ys.append(Y)
  Zs.append(Z)
print(Xs)
dict = {
    'X' : Xs,
    'Y' : Ys,
    'Z' : Zs
}
df = pd.DataFrame(dict,columns = ['X','Y','Z'])
df
##dd09

[-1136068.1830873997, -1135882.947422289, -1136209.8899883402, -1136088.4497600892]


,X,Y,Z
0,-1.136068e+06,6.091187e+06,1.507226e+06
1,-1.135883e+06,6.091203e+06,1.507300e+06
2,-1.136210e+06,6.091139e+06,1.507311e+06
3,-1.136088e+06,6.091289e+06,1.506804e+06


In [33]:
e = df.to_csv('gcp09.csv',index=False)